In [1]:
import pandas as pd
import numpy as np

import os

def load_hotel_reserve():
  customer_tb = pd.read_csv('./data/customer.csv')
  hotel_tb = pd.read_csv('./data/hotel.csv')
  reserve_tb = pd.read_csv('./data/reserve.csv')
  return customer_tb, hotel_tb, reserve_tb


def load_holiday_mst():
  holiday_tb = pd.read_csv('./data/holiday_mst.csv',
                           index_col=False)
  return holiday_tb


def load_production():
  production_tb = pd.read_csv('./data/production.csv')
  return production_tb


def load_production_missing_num():
  production_tb = pd.read_csv('./data/production_missing_num.csv')
  return production_tb


def load_production_missing_category():
  production_tb = pd.read_csv('./data/production_missing_category.csv')
  return production_tb


def load_monthly_index():
  monthly_index_tb = \
    pd.read_csv('./data/monthly_index.csv')
  return monthly_index_tb


def load_meros_txt():
  with open('./data/txt/meros.txt', 'r') as f:
    meros = f.read()
    f.close()
  return meros


In [2]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

print(reserve_tb)

     reserve_id hotel_id customer_id     reserve_datetime checkin_date  \
0            r1     h_75         c_1  2016-03-06 13:09:42   2016-03-26   
1            r2    h_219         c_1  2016-07-16 23:39:55   2016-07-20   
2            r3    h_179         c_1  2016-09-24 10:03:17   2016-10-19   
3            r4    h_214         c_1  2017-03-08 03:20:10   2017-03-29   
4            r5     h_16         c_1  2017-09-05 19:50:37   2017-09-22   
...         ...      ...         ...                  ...          ...   
4025      r4026    h_129       c_999  2017-06-27 23:00:02   2017-07-10   
4026      r4027     h_97       c_999  2017-09-29 05:24:57   2017-10-09   
4027      r4028     h_27       c_999  2018-03-14 05:01:45   2018-04-02   
4028      r4029     h_48      c_1000  2016-04-16 15:20:17   2016-05-10   
4029      r4030    h_117      c_1000  2016-06-06 08:16:51   2016-07-06   

     checkin_time checkout_date  people_num  total_price  
0        10:00:00    2016-03-29           4        9

#### 2. 분석 단위를 손실 없이 변경하기

손실없이 데이터 분석 단위 변경 : Aggregation

-> 데이터의 가치를 되도록 손실없이 압축하여 데이터의 단위(행의 의미)를 변환할 수 있는 처리

-> 의미 : 데이터의 전체적인 경향을 파악하는데 도움이 되고 정보 손실이 적은 변환 처리가 가능

-> 방법 : GROUP_BY로 집약할 단위를 지정하여 count, sum 함수를 이용하는 방법

- agg() 함수를 활용한 개수 산출

In [3]:
# agg 함수의 매개변수에 dictionary 오브젝트를 지정하여 Aggregation 처리를 한꺼번에 지정 가능
# dictionary 오브젝트는 key에 열의 이름, value에 집약함수 이름을 지정
result = reserve_tb\
    .groupby('hotel_id')\
    .agg({'reserve_id': 'count', 'customer_id': 'nunique'})

In [4]:
result.reset_index(inplace=True)
result.columns = ['hotel_id', 'rsv_cnt', 'cus_cnt']
result

,hotel_id,rsv_cnt,cus_cnt
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13
...,...,...,...
295,h_95,13,13
296,h_96,13,13
297,h_97,16,16
298,h_98,17,16


- sum()을 이용한 매출 합계 산출

In [5]:
# 분석 대상의 값이 숫자일때 데이터의 합을 계산하는 경우에 사용
result = reserve_tb\
    .groupby(['hotel_id', 'people_num'])['total_price']\
    .sum().reset_index()

In [6]:
result.rename(columns={'total_price': 'price_sum'}, inplace=True)
result

,hotel_id,people_num,price_sum
0,h_1,1,156600
1,h_1,2,156600
2,h_1,3,391500
3,h_1,4,417600
4,h_10,1,11200
...,...,...,...
1154,h_98,3,793800
1155,h_98,4,453600
1156,h_99,1,179200
1157,h_99,2,448000


ex) 1번 행: h_1 호텔에서 2명이 투숙한 모든 예약의 가격을 합하면 156600원  
    3번 행: h_1 호텔에서 4명이 투숙한 모든 예약의 가격을 합하면 417600원

- max, min, mean, median, percentile(백분위수) 산출

In [7]:
# 분석대상의 값이 숫자일때 최대값, 최소값, 평균, 중간값, 백분위수(하위 20%)를 계산하는 경우에 사용
result = reserve_tb\
    .groupby('hotel_id')\
    .agg({'total_price': ['max', 'min', 'mean', 'median',
                          lambda x: np.percentile(x, q=20)]})\
    .reset_index()

In [8]:
result.columns = ['hotel_id', 'price_max', 'price_min', 'price_mean', 'price_median', 'price_20per']
result

,hotel_id,price_max,price_min,price_mean,price_median,price_20per
0,h_1,208800,26100,112230.000000,104400.0,73080.0
1,h_10,67200,11200,42933.333333,50400.0,26880.0
2,h_100,57600,4800,27600.000000,28800.0,9600.0
3,h_101,168000,14000,75764.705882,56000.0,30800.0
4,h_102,72000,12000,32769.230769,24000.0,18000.0
...,...,...,...,...,...,...
295,h_95,518400,43200,275815.384615,259200.0,146880.0
296,h_96,66600,7400,33015.384615,29600.0,17760.0
297,h_97,250800,20900,83600.000000,62700.0,20900.0
298,h_98,226800,18900,96723.529412,75600.0,56700.0


- Variance(분산)과 Standard deviation(표준편차) 산출

In [9]:
# 분산값과 표준편차값은 데이터의 분포 정도를 나타냄
result = reserve_tb\
    .groupby('hotel_id')\
    .agg({'total_price': ['var', 'std']}).reset_index()

In [10]:
result.columns = ['hotel_id', 'price_var', 'price_std']
pd.options.display.float_format = '{:,.0f}'.format
result.fillna(0, inplace=True)
result

,hotel_id,price_var,price_std
0,h_1,"3,186,549,000","56,450"
1,h_10,"825,813,333","28,737"
2,h_100,"319,831,579","17,884"
3,h_101,"2,402,441,176","49,015"
4,h_102,"357,692,308","18,913"
...,...,...,...
295,h_95,"33,137,723,077","182,038"
296,h_96,"315,923,077","17,774"
297,h_97,"5,474,685,333","73,991"
298,h_98,"3,432,893,162","58,591"


- mode(최빈값) 계산

In [11]:
# 최빈값은 가장 많이 나타나는 수치를 의미
# Reserve_tb에서 최빈값 계산
reserve_tb['total_price'].round(-3).mode()      # -3은 천단위로 반올림

0    10000
1    20000
2    40000
Name: total_price, dtype: int64

- rank()를 이용한 순위 계산

In [12]:
# 그룹별로 순서를 정렬하고 순위를 매겨서 새로 추가된 열에 기록
reserve_tb['reserve_datetime'] = pd.to_datetime(
    reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')

In [13]:
reserve_tb['log_no'] = reserve_tb\
    .groupby('customer_id')['reserve_datetime']\
    .rank(ascending=True, method='first')   # 오름차순, 동일한 시간이면 DF 순서대로 순위 매김
reserve_tb

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log_no
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,1
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,2
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,4
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,5
...,...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,4
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,5
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800,6
4028,r4029,h_48,c_1000,2016-04-16 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000,1


In [14]:
# 예약 테이블을 이용하여 호텔별 예약 건수에 따른 순위를 매김
rsv_cnt_tb = reserve_tb.groupby('hotel_id').size().reset_index()
rsv_cnt_tb.columns = ['hotel_id', 'rsv_cnt']

In [15]:
rsv_cnt_tb['rsv_cnt_rank'] = rsv_cnt_tb['rsv_cnt']\
    .rank(ascending=False, method='min')  # 내림차순, 동점이면 다음 순위 건너뜀

In [16]:
rsv_cnt_tb.drop('rsv_cnt', axis=1, inplace=True)
rsv_cnt_tb

,hotel_id,rsv_cnt_rank
0,h_1,235
1,h_10,300
2,h_100,12
3,h_101,43
4,h_102,139
...,...,...
295,h_95,139
296,h_96,139
297,h_97,60
298,h_98,43
